In [1]:
!pip install bottle

In [3]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/olaszpakowska/Desktop/studia/magisterka-uw/Python-SQL/museum-management-system


In [4]:
from bottle import Bottle, run, template, static_file, request, redirect, response
import sqlite3

app = Bottle()

# Base directory for the project
BASE_DIR =  os.getcwd()

# Static directories
STATIC_DIR = os.path.join(BASE_DIR, "static")

print("Static directory:", STATIC_DIR)


Static directory: /Users/olaszpakowska/Desktop/studia/magisterka-uw/Python-SQL/museum-management-system/static


In [5]:
@app.route('/static/<filepath:path>')
def serve_static(filepath):
    return static_file(filepath, root=STATIC_DIR)

In [6]:
def create_database():
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()
    
    cursor.executescript('''
    CREATE TABLE IF NOT EXISTS Visitors (
        visitor_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        phone INTEGER,
        password TEXT NOT NULL
    );

    CREATE TABLE IF NOT EXISTS Exhibits (
        exhibit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        description TEXT,
        start_date DATE,
        end_date DATE,
        popularity INTEGER,
        event_id INTEGER,
        staff_id INTEGER,
        FOREIGN KEY (event_id) REFERENCES Events(event_id),
        FOREIGN KEY (staff_id) REFERENCES Staff(staff_id)
    );

    CREATE TABLE IF NOT EXISTS Tickets (
        ticket_id INTEGER PRIMARY KEY AUTOINCREMENT,
        visitor_id INTEGER,
        exhibit_id INTEGER,
        purchase_date DATE,
        price INTEGER,
        FOREIGN KEY (visitor_id) REFERENCES Visitors(visitor_id),
        FOREIGN KEY (exhibit_id) REFERENCES Exhibits(exhibit_id)
    );

    CREATE TABLE IF NOT EXISTS Events (
        event_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        description TEXT,
        date DATE,
        time TIME
    );

    CREATE TABLE IF NOT EXISTS Staff (
        staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        role TEXT
    );

    CREATE TRIGGER update_exhibit_popularity
    AFTER INSERT ON Tickets
    BEGIN
        UPDATE Exhibits
        SET popularity = (
            SELECT COUNT(*)
            FROM Tickets
            WHERE Tickets.exhibit_id = Exhibits.exhibit_id
        )
        WHERE Exhibits.exhibit_id = NEW.exhibit_id;
    END;

    CREATE VIEW staff_role_summary AS
    SELECT role, COUNT(*) AS role_count
    FROM Staff
    GROUP BY role;

    ''')


    conn.commit()
    conn.close()
    print("Database schema, trigger and view created successfully!")

create_database()

OperationalError: trigger update_exhibit_popularity already exists

In [7]:
def insert_manual_data():
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    # Create indexes for optimized performance
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_visitor_id ON Visitors (visitor_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_staff_id ON Staff (staff_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_event_id ON Events (event_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_exhibit_id ON Exhibits (exhibit_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_ticket_id ON Tickets (ticket_id);")
    print("Indexes created successfully.")

    # Insert Visitors
    visitors = [
        (1, "Alice Johnson", "alice@example.com", 1234567890, "password123"),
        (2, "Bob Smith", "bob@example.com", 1234567891, "password123"),
        (3, "Charlie Brown", "charlie@example.com", 1234567892, "password123"),
        (4, "Diana Ross", "diana@example.com", 1234567893, "password123"),
        (5, "Ethan Hunt", "ethan@example.com", 1234567894, "password123")
    ]
    cursor.executemany("INSERT INTO Visitors (visitor_id, name, email, phone, password) VALUES (?, ?, ?, ?, ?)", visitors)

    # Insert Staff
    staff = [
        (1, "John Doe", "Curator"),
        (2, "Jane Smith", "Guide"),
        (3, "Emily Davis", "Administrator"),
        (4, "Chris Johnson", "Technician"),
        (5, "Laura Wilson", "Guide"),
        (6, "Michael Brown", "Technician"),
        (7, "Sarah Miller", "Guide"),
        (8, "David Lee", "Curator"),
        (9, "Sophia Garcia", "Administrator"),
        (10, "James Anderson", "Security"),
        (11, "Olivia Martinez", "Guide"),
        (12, "Liam Thompson", "Technician"),
        (13, "Emma White", "Receptionist"),
        (14, "Benjamin Taylor", "Technician"),
        (15, "Charlotte Moore", "Curator"),
        (16, "Lucas Martin", "Security"),
        (17, "Amelia Hall", "Guide"),
        (18, "Henry Allen", "Administrator"),
        (19, "Ava Young", "Technician"),
        (20, "Ella King", "Guide"),
        (21, "Alexander Wright", "Technician"),
        (22, "Mia Scott", "Guide"),
        (23, "Ethan Adams", "Curator"),
        (24, "Harper Baker", "Administrator"),
        (25, "Daniel Nelson", "Guide"),
        (26, "Abigail Perez", "Security"),
        (27, "Matthew Rogers", "Technician"),
        (28, "Isabella Reed", "Guide"),
        (29, "Jackson Murphy", "Curator"),
        (30, "Sofia Bell", "Administrator")
    ]

    cursor.executemany("INSERT INTO Staff (staff_id, name, role) VALUES (?, ?, ?)", staff)

    # Insert Events
    events = [
        (1, "Art Workshop", "Learn painting techniques with professionals.", "2025-01-25", "10:00"),
        (2, "History Talk", "Explore the history behind famous artifacts.", "2025-02-10", "15:00"),
        (3, "Sculpture Exhibition", "A special exhibition of sculptures.", "2025-02-25", "13:00"),
        (4, "Photography Seminar", "Improve your photography skills.", "2025-03-05", "11:00"),
        (5, "Ancient Artifacts", "Discover artifacts from ancient civilizations.", "2025-04-01", "14:00")
    ]
    cursor.executemany("INSERT INTO Events (event_id, name, description, date, time) VALUES (?, ?, ?, ?, ?)", events)

    # Insert Exhibits
    exhibits = [
        (1, "Ancient Egypt", "Artifacts from ancient Egypt.", "2025-01-01", "2025-12-31", 1, 1),
        (2, "Renaissance Art", "Paintings and sculptures from the Renaissance.", "2025-02-01", "2025-10-31", 2, 2),
        (3, "Modern Art", "Contemporary art pieces from the 20th century.", "2025-03-01", "2025-11-30", 3, 3),
        (4, "Abstract Art", "Exploring abstract art movements.", "2025-04-01", "2025-09-30", 4, 4),
        (5, "Impressionism", "The evolution of impressionist art.", "2025-05-01", "2025-08-31", 5, 5)
    ]
    cursor.executemany("INSERT INTO Exhibits (exhibit_id, name, description, start_date, end_date, event_id, staff_id) VALUES (?, ?, ?, ?, ?, ?, ?)", exhibits)

    # Insert Tickets
    tickets = [
        (1, 1, 1, "2025-01-15", 20),
        (2, 2, 2, "2025-02-20", 25),
        (3, 3, 3, "2025-03-10", 15),
        (4, 4, 4, "2025-04-05", 30),
        (5, 5, 5, "2025-05-12", 18)
    ]
    cursor.executemany("INSERT INTO Tickets (ticket_id, visitor_id, exhibit_id, purchase_date, price) VALUES (?, ?, ?, ?, ?)", tickets)

    conn.commit()
    conn.close()
    print("Manual data inserted successfully!")

insert_manual_data()



Indexes created successfully.


IntegrityError: UNIQUE constraint failed: Visitors.visitor_id

In [8]:
def verify_manual_data():
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    print("Visitors:")
    cursor.execute("SELECT * FROM Visitors")
    print(cursor.fetchall())

    print("\nStaff:")
    cursor.execute("SELECT * FROM Staff")
    print(cursor.fetchall())

    print("\nEvents:")
    cursor.execute("SELECT * FROM Events")
    print(cursor.fetchall())

    print("\nExhibits:")
    cursor.execute("SELECT * FROM Exhibits")
    print(cursor.fetchall())

    print("\nTickets:")
    cursor.execute("SELECT * FROM Tickets")
    print(cursor.fetchall())

    conn.close()

verify_manual_data()

Visitors:
[(1, 'Alice Johnson', 'alice@example.com', 1234567890, 'password123'), (2, 'Bob Smith', 'bob@example.com', 1234567891, 'password123'), (3, 'Charlie Brown', 'charlie@example.com', 1234567892, 'password123'), (4, 'Diana Ross', 'diana@example.com', 1234567893, 'password123'), (5, 'Ethan Hunt', 'ethan@example.com', 1234567894, 'password123'), (6, 'ola', 'ola@gmail.com', None, 'ola')]

Staff:
[(1, 'John Doe', 'Curator'), (2, 'Jane Smith', 'Guide'), (3, 'Emily Davis', 'Administrator'), (4, 'Chris Johnson', 'Technician'), (5, 'Laura Wilson', 'Guide'), (6, 'Michael Brown', 'Technician'), (7, 'Sarah Miller', 'Guide'), (8, 'David Lee', 'Curator'), (9, 'Sophia Garcia', 'Administrator'), (10, 'James Anderson', 'Security'), (11, 'Olivia Martinez', 'Guide'), (12, 'Liam Thompson', 'Technician'), (13, 'Emma White', 'Receptionist'), (14, 'Benjamin Taylor', 'Technician'), (15, 'Charlotte Moore', 'Curator'), (16, 'Lucas Martin', 'Security'), (17, 'Amelia Hall', 'Guide'), (18, 'Henry Allen', 'Ad

In [9]:
@app.route('/')
def home():
    # Connect to the database
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    # Query the staff role summary view
    cursor.execute("SELECT role, role_count FROM staff_role_summary;")
    staff_data = cursor.fetchall()  # Fetch the role data
    conn.close()

    # Helper function for pluralization
    def pluralize(role, count):
        if count > 1:
            if role.endswith("y"):
                return role[:-1] + "ies"  # Change 'y' to 'ies'
            else:
                return role + "s"  # Add 's' for regular pluralization
        return role  # No change for singular

    # Calculate the total staff count and create the message
    total_staff = sum(row[1] for row in staff_data)
    staff_details = "".join([
        f"<p>- {row[1]} {pluralize(row[0], row[1])}</p>"
        for row in staff_data
    ])
    message = f"<p>{total_staff} people are working to deliver you the best cultural experience:</p>{staff_details}"

    # Check login status
    username = request.get_cookie("username", secret='secret_key')
    if username:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
            <a href="/logout">Logout</a>
        </nav>
        """
        greeting = f"<h1>Welcome back, {username}!</h1>"
    else:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        """
        greeting = "<h1>Welcome to the Museum!</h1>"

    # Render the HTML with the message
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Museum Management System</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }}
            header {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            nav {{
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }}
            nav a {{
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }}
            nav a:hover {{
                text-decoration: underline;
            }}
            main {{
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: 80%;
            }}
            footer {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            p {{
                margin: 0.5rem 0;
                font-size: 1.2rem;
            }}
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
        {nav_links}
        <main>
            {greeting}
            <h2>Explore the world of art and history</h2>
            <p>Welcome to our museum! Explore exhibits, join events, and embark on a journey through time and culture.</p></br>
            {message}
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html


In [10]:
@app.route('/register', method=['GET', 'POST'])
def register():
    if request.method == 'POST':
        name = request.forms.get('name')
        email = request.forms.get('email')
        password = request.forms.get('password')

        # Insert the user into the database
        conn = sqlite3.connect('museum.db')
        cursor = conn.cursor()
        try:
            cursor.execute("INSERT INTO Visitors (name, email, password) VALUES (?, ?, ?)", (name, email, password))
            conn.commit()
            conn.close()
            return "<h1>Registration successful! <a href='/login'>Login here</a>.</h1>"
        except sqlite3.IntegrityError:
            conn.close()
            return "<h1>Email already exists. <a href='/register'>Try again</a>.</h1>"

    # Display registration form
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Login</title>
         <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }
            header {
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }
            nav {
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }
            nav a {
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }
            nav a:hover {
                text-decoration: underline;
            }
            main {
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: 80%;
            }
            footer {
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
        <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        <main>
        <form action="/register" method="post">
            <label>Name:</label>
            <input type="text" name="name" required><br><br>
            <label>Email:</label>
            <input type="email" name="email" required><br><br>
            <label>Password:</label>
            <input type="password" name="password" required><br><br>
            <button type="submit">Register</button>
        </form>
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html

In [11]:
# Login route
@app.route('/login', method=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.forms.get('email')
        password = request.forms.get('password')

        # Validate user credentials
        conn = sqlite3.connect('museum.db')
        cursor = conn.cursor()
        cursor.execute("SELECT visitor_id, name FROM Visitors WHERE email = ? AND password = ?", (email, password))
        user = cursor.fetchone()
        conn.close()

        if user:
            # Set session cookie
            response.set_cookie("visitor_id", str(user[0]), secret='secret_key')
            response.set_cookie("username", user[1], secret='secret_key')
            return redirect('/')
        else:
            return "<h1>Invalid email or password. <a href='/login'>Try again</a>.</h1>"

    # Display login form
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Login</title>
         <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }
            header {
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }
            nav {
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }
            nav a {
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }
            nav a:hover {
                text-decoration: underline;
            }
            main {
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: 80%;
            }
            footer {
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
         <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        <main>
            <h2>Login Page</h2>
            <form action="/login" method="post">
                <label for="email">Email:</label>
                <input type="text" id="email" name="email"><br><br>
                <label for="password">Password:</label>
                <input type="password" id="password" name="password"><br><br>
                <button type="submit">Login</button>
            </form>
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html


In [12]:
# Logout route
@app.route('/logout')
def logout():
    # Clear session cookies
    response.delete_cookie("visitor_id", secret='secret_key')
    response.delete_cookie("username", secret='secret_key')
    return redirect('/')

# Example of a protected route
@app.route('/protected')
def protected():
    visitor_id = request.get_cookie("visitor_id", secret='secret_key')
    username = request.get_cookie("username", secret='secret_key')
    if not visitor_id:
        return redirect('/login')  # Redirect to login if not logged in
    return f"<h1>Welcome back, {username}! This is a protected page.</h1>"

In [13]:
import json
from flask import render_template
@app.route('/exhibits')
def exhibits_popularity():
    # Connect to the database
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    # Fetch exhibits and their ticket counts
    cursor.execute("""
        SELECT Exhibits.name, COUNT(Tickets.ticket_id) AS ticket_count
        FROM Exhibits
        LEFT JOIN Tickets ON Exhibits.exhibit_id = Tickets.exhibit_id
        GROUP BY Exhibits.exhibit_id
        ORDER BY ticket_count DESC;
    """)
    exhibits = cursor.fetchall()

        # Subquery to find exhibits with above-average ticket sales
    cursor.execute('''
        SELECT name
        FROM Exhibits
        WHERE exhibit_id IN (
            SELECT exhibit_id
            FROM Tickets
            GROUP BY exhibit_id
            HAVING COUNT(ticket_id) > (
                SELECT AVG(tickets_bought)
                FROM (
                    SELECT COUNT(ticket_id) AS tickets_bought
                    FROM Tickets
                    GROUP BY exhibit_id
                )
            )
        );
    ''')
    above_average_exhibits = cursor.fetchall()
    conn.close()

    # Generate the sentence dynamically
    if above_average_exhibits:
        above_average_sentence = (
            "Exhibits with above-average ticket sales: "
            + ", ".join([exhibit[0] for exhibit in above_average_exhibits])
        )
    else:
        above_average_sentence = "No exhibits have above-average ticket sales at the moment."
    
    # Extract data for the chart
    exhibit_names = [exhibit[0] for exhibit in exhibits]
    ticket_counts = [exhibit[1] for exhibit in exhibits]

    # Convert data to JSON for JavaScript
    import json
    exhibit_names_js = json.dumps(exhibit_names)
    ticket_counts_js = json.dumps(ticket_counts)
    
    table_rows = ""
    for exhibit in exhibits:
        name, ticket_count = exhibit
        table_rows += f"""
        <tr>
            <td>{name}</td>
            <td>{ticket_count}</td>
        </tr>
        """

     # Check login status
    username = request.get_cookie("username", secret='secret_key')
    if username:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
            <a href="/logout">Logout</a>
        </nav>
        """
    else:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        """
    
    # HTML structure
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Exhibits Popularity</title>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }}
            header {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            nav {{
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }}
            nav a {{
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }}
            nav a:hover {{
                text-decoration: underline;
            }}
            main {{
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: auto;
                overflow: auto;
            }}
            table {{
                width: 50%;
                border-collapse: collapse;
                margin: 2rem auto;
            }}
            th, td {{
                border: 1px solid white;
                padding: 8px;
                text-align: center;
            }}
            th {{
                background-color: rgba(0, 0, 0, 0.8);
                color: white;
            }}
            footer {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            #chart-container {{
                width: 600px; /* Adjust as needed */
                height: 400px; /* Adjust as needed */
                margin: 0 auto;
            }}
            canvas {{
                display: block;
                width: 100%; /* Matches parent width */
                height: 100%; /* Matches parent height */
            }}

        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
         {nav_links}
        <main>
            <h2>Exhibits Popularity</h2>
            <p>{above_average_sentence}</p>
            <table>
                <thead>
                    <tr>
                        <th>Exhibit</th>
                        <th>Popularity</th>
                    </tr>
                </thead>
                <tbody>
                    {table_rows}
                </tbody>
            </table>
            <div id="chart-container">
                <canvas id="ticketChart"></canvas>
            </div>
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
        <script>
            // Chart data
            const exhibitNames = {exhibit_names_js};
            const ticketCounts = {ticket_counts_js};

            // Render the chart
            const ctx = document.getElementById('ticketChart').getContext('2d');
            new Chart(ctx, {{
                type: 'bar',
                data: {{
                    labels: exhibitNames,
                    datasets: [{{
                        label: 'Tickets bought', 
                        data: ticketCounts,
                        backgroundColor: 'rgba(75, 192, 192, 0.9)',
                        borderColor: 'rgba(75, 192, 192, 1)',
                        borderWidth: 1
                    }}]
                }},
              options: {{
                    responsive: true,
                    plugins: {{
                        legend: {{
                            position: 'top',
                            labels: {{
                                color: 'white'
                            }}
                        }},
                        tooltip: {{
                            bodyColor: 'white',
                            titleColor: 'white',
                            backgroundColor: 'rgba(0, 0, 0, 0.7)'
                        }}
                    }},
                    scales: {{
                        x: {{
                            ticks: {{
                                color: 'white'
                            }},
                            grid: {{
                                color: 'rgba(255, 255, 255, 0.3)'
                            }}
                        }},
                        y: {{
                            ticks: {{
                                color: 'white'
                            }},
                            grid: {{
                                color: 'rgba(255, 255, 255, 0.3)'
                            }}
                        }}
                    }}
                }}
            }});
        </script>
    </body>
    </html>
    """
    return html



In [14]:
@app.route('/tickets', method=['GET', 'POST'])
def tickets():
    if request.method == 'POST':
        # Handle ticket purchase
        # Get visitor_id from the cookie
        visitor_id = request.get_cookie("visitor_id", secret='secret_key')
        if not visitor_id:
            return "<h1>You must be logged in to purchase tickets. <a href='/login'>Login</a></h1>"

        exhibit_id = request.forms.get('exhibit_id')
        age_group = request.forms.get('age_group')
        purchase_date = request.forms.get('purchase_date')

        # Calculate ticket price based on age group
        price_mapping = {"0-3": 0, "4-10": 5, "11-50": 10, "50+": 1}
        ticket_price = price_mapping.get(age_group, 0)

        # Insert the ticket into the database
        conn = sqlite3.connect('museum.db')
        cursor = conn.cursor()
        try:
            cursor.execute("""
                INSERT INTO Tickets (visitor_id, exhibit_id, purchase_date, price)
                VALUES (?, ?, ?, ?)
            """, (visitor_id, exhibit_id, purchase_date, ticket_price))
            conn.commit()
            conn.close()
            return "<h1>Ticket purchased successfully! <a href='/'>Go back to homepage</a>.</h1>"
        except Exception as e:
            conn.close()
            return f"<h1>Error purchasing ticket: {str(e)}. <a href='/tickets'>Try again</a>.</h1>"

    # For GET request, display ticket options
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    # Fetch available purchase dates
    cursor.execute("SELECT DISTINCT purchase_date FROM Tickets")
    purchase_dates = cursor.fetchall()

    # Fetch exhibits
    cursor.execute("SELECT exhibit_id, name FROM Exhibits")
    exhibits = cursor.fetchall()
    conn.close()

     # Check login status
    username = request.get_cookie("username", secret='secret_key')
    if username:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
            <a href="/logout">Logout</a>
        </nav>
        """
    else:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        """
    
    # HTML structure
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Tickets</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }}
            header {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            nav {{
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }}
            nav a {{
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }}
            nav a:hover {{
                text-decoration: underline;
            }}
            main {{
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: auto;
            }}
            footer {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            table {{
                width: 70%;
                margin: 2rem auto;
                border-collapse: collapse;
                color: white;
            }}
            th, td {{
                border: 1px solid white;
                padding: 10px;
                text-align: center;
            }}
            th {{
                background-color: rgba(0, 0, 0, 0.8);
            }}
            .form-container {{
                margin: 2rem auto;
                width: 70%;
                padding: 1rem;
                background-color: rgba(0, 0, 0, 0.8);
                border-radius: 10px;
            }}
            .form-container label, select, input {{
                display: block;
                margin: 1rem auto;
                color: white;
                width: 80%;
                font-size: 1rem;
            }}
            .form-container button {{
                padding: 0.5rem 1rem;
                font-size: 1rem;
                margin: 1rem auto;
                background-color: rgba(75, 192, 192, 0.9);
                color: white;
                border: none;
                cursor: pointer;
                border-radius: 5px;
            }}
            .form-container button:hover {{
                background-color: rgba(75, 192, 192, 1);
            }}
            select, option, input[type="date"] {{
                color: black; /* Change dropdown and option text color to black */
                background-color: white; /* Ensure a contrasting background */
            }}
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
         {nav_links}
        <main>
            <h2>Ticket Options</h2>
            <div class="form-container">
                <form method="POST">
                    <label for="exhibit_id">Select Exhibit:</label>
                    <select name="exhibit_id" required>
                        {"".join([f'<option value="{exhibit[0]}">{exhibit[1]}</option>' for exhibit in exhibits])}
                    </select>
                    <label for="age_group">Select Age Group:</label>
                    <select name="age_group" required>
                        <option value="0-3">0-3 (Free)</option>
                        <option value="4-10">4-10 (5 EURO)</option>
                        <option value="11-50">11-50 (10 EURO)</option>
                        <option value="50+">50+ (1 EURO)</option>
                    </select>
                    <label for="purchase_date">Purchase Date:</label>
                    <input type="date" name="purchase_date" required>
                    <button type="submit">Purchase Ticket</button>
                </form>
            </div>
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html


In [15]:
from datetime import datetime, timedelta
import calendar

@app.route('/events')
def events_calendar():
    # Connect to the database
    conn = sqlite3.connect('museum.db')
    cursor = conn.cursor()

    # Get current date and calculate next month's date
    current_date = datetime.now()
    first_of_next_month = (current_date.replace(day=1) + timedelta(days=32)).replace(day=1)

    # Query for events in the current and next month
    cursor.execute("""
        SELECT name, description, date, time 
        FROM Events 
        WHERE date >= ? AND date < ?
        ORDER BY date, time
    """, (current_date.strftime('%Y-%m-%d'), (first_of_next_month + timedelta(days=32)).strftime('%Y-%m-%d')))
    events = cursor.fetchall()
    conn.close()

    # Separate events into current and next month
    current_month_events = []
    next_month_events = []
    for event in events:
        event_date = datetime.strptime(event[2], '%Y-%m-%d')
        if event_date.month == current_date.month:
            current_month_events.append(event)
        elif event_date.month == first_of_next_month.month:
            next_month_events.append(event)

    # Generate calendar rows for each month
    def generate_calendar_rows(events, month, year):
        cal = calendar.Calendar()
        days_in_month = cal.itermonthdays2(year, month)  # Generate days with weekday info (0=Mon, 6=Sun)

        rows = ""
        current_week = ""
        for day, weekday in days_in_month:
            if day == 0:  # Empty cell for days outside the month
                current_week += '<td class="empty"></td>'
            else:
                # Check if there's an event on this day
                day_events = [e for e in events if datetime.strptime(e[2], '%Y-%m-%d').day == day]
                event_html = "<br>".join([f"<div class='event'>{e[0]}</div>" for e in day_events])
                current_week += f'<td>{day}{event_html}</td>'
            if weekday == 6:  # End of the week
                rows += f"<tr>{current_week}</tr>"
                current_week = ""
        if current_week:  # Add remaining days
            rows += f"<tr>{current_week}</tr>"
        return rows

    # Generate calendar tables for both months
    current_month_table = generate_calendar_rows(current_month_events, current_date.month, current_date.year)
    next_month_table = generate_calendar_rows(next_month_events, first_of_next_month.month, first_of_next_month.year)

     # Check login status
    username = request.get_cookie("username", secret='secret_key')
    if username:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
            <a href="/logout">Logout</a>
        </nav>
        """
    else:
        nav_links = """
        <nav>
            <a href="/">Home</a>
            <a href="/register">Register</a>
            <a href="/login">Login</a>
            <a href="/tickets">Tickets</a>
            <a href="/exhibits">Exhibits Popularity</a>
            <a href="/events">Event Calendar</a>
        </nav>
        """
    
    # HTML structure
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Event Calendar</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-image: url('/static/images/museum.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                background-position: center;
                height: 100vh;
                display: flex;
                flex-direction: column;
                justify-content: space-between;
                color: white;
            }}
            header {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
            nav {{
                background-color: rgba(0, 0, 0, 0.8);
                text-align: center;
                padding: 0.5rem;
            }}
            nav a {{
                color: white;
                text-decoration: none;
                margin: 0 15px;
                font-weight: bold;
            }}
            nav a:hover {{
                text-decoration: underline;
            }}
            main {{
                flex: 1;
                padding: 2rem;
                text-align: center;
                background-color: rgba(0, 0, 0, 0.6);
                margin: 0 auto;
                width: 90%;
                height: auto;
                border-radius: 10px;
                overflow: auto;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 1rem auto;
                color: white;
            }}
            th, td {{
                border: 1px solid white;
                padding: 8px;
                text-align: center;
                vertical-align: top;
            }}
            th {{
                background-color: rgba(0, 0, 0, 0.8);
            }}
            .empty {{
                background-color: rgba(255, 255, 255, 0.2);
            }}
            .event {{
                background-color: rgba(75, 192, 192, 0.7);
                border-radius: 5px;
                padding: 3px;
                margin-top: 5px;
            }}
            footer {{
                background-color: rgba(0, 0, 0, 0.9);
                color: white;
                text-align: center;
                padding: 1rem 0;
            }}
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to the Museum</h1>
        </header>
        {nav_links}
        <main>
            <h2>Event Calendar</h2>
            <table>
                <thead>
                    <tr><th colspan="7">This Month</th></tr>
                    <tr>
                        <th>Mon</th><th>Tue</th><th>Wed</th><th>Thu</th><th>Fri</th><th>Sat</th><th>Sun</th>
                    </tr>
                </thead>
                <tbody>
                    {current_month_table}
                </tbody>
            </table>
            <table>
                <thead>
                    <tr><th colspan="7">Next Month</th></tr>
                    <tr>
                        <th>Mon</th><th>Tue</th><th>Wed</th><th>Thu</th><th>Fri</th><th>Sat</th><th>Sun</th>
                    </tr>
                </thead>
                <tbody>
                    {next_month_table}
                </tbody>
            </table>
        </main>
        <footer>
            <p>&copy; 2025 Museum Management System. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html


In [16]:
def start_app():
    run(app, host="localhost", port=8080, debug=True)

import threading
thread = threading.Thread(target=start_app, daemon=True)
thread.start()

print("Bottle app is running at http://localhost:8080")

Bottle app is running at http://localhost:8080


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [21/Jan/2025 17:19:37] "GET / HTTP/1.1" 200 2939
127.0.0.1 - - [21/Jan/2025 17:19:39] "GET /register HTTP/1.1" 200 2774
127.0.0.1 - - [21/Jan/2025 17:19:40] "GET /login HTTP/1.1" 200 2768
127.0.0.1 - - [21/Jan/2025 17:19:41] "GET /tickets HTTP/1.1" 200 5080
127.0.0.1 - - [21/Jan/2025 17:19:41] "GET /exhibits HTTP/1.1" 200 6306
127.0.0.1 - - [21/Jan/2025 17:19:42] "GET /events HTTP/1.1" 200 5071
